# STEP 2: 2020 Presidential and Vice Presidential Debate Consolidator

## Polisticians Semester Project

### data sourced from Rev.com; these data were easier to wrangle than the CPD's

In [6]:
# import packages

import re
import pandas as pd
import time

In [10]:
# File used to open 2020 debate text files and prepare them for integration w/
# the rest of the process

t_0 = time.time()
print('Start the process')

# file names

# these data weren't scraped since there are so few transcripts
# the text files below were created via copying from the source data as large blocks of text

d = ['2020_debate1.txt', '2020_debate2.txt', '2020_VP_debate.txt'] 

# list to which transcripts will be added
t = []

# blank DF with transcript header; will add the transcript text to the this DF
df = pd.DataFrame(columns = ['Transcript'])

# open file, split the text into a list by tab (4 spaces in the file means a transition to a new speaker)
with open('2020_debate1.txt','r') as f:    
    for l in f:
        t = re.split(r'\s{4,}', l) # split text into blocks by 4 spaces to obtain transitions 

# add list to DF
df['Transcript'] = t

print('First Presidential debate pulled') # let the user know the first debate was pulled

# add extra info to DF, like which debate and the position (P or VP)
df['Debate'] = 'September 29, 2020 Debate'
df['Position'] = 'President'

# start same process for the second Prez debate; blank df with Transcript header
df2 = pd.DataFrame(columns = ['Transcript'])

t = [] # blank list to hold transcripts

with open('2020_debate2.txt','r') as f: # open second text file
    for l in f:
        t = re.split(r'\s{4,}', l) # same logic to pull blocks of text

df2['Transcript'] = t # add transcript blocks to the DF
        
print('Second Presidential debate pulled')       

df2['Debate'] = 'October 22, 2020 Debate' # Add information
df2['Position'] = 'President'

# start the process for the VP debate; process is the same as the two Prez
# debates
df3 = pd.DataFrame(columns = ['Transcript'])

with open('2020_VP_debate.txt','r') as f:    
    for l in f:
        t = re.split(r'\s{4,}', l)
 
print('Only Vice Presidential debate pulled')

df3['Transcript'] = t

df3['Debate'] = 'October 7, 2020 Debate'
df3['Position'] = 'Vice President'

df = df.append(df2) # append the DFs created above
df = df.append(df3)

print('Finished in {0: .2f} seconds'.format(time.time()-t_0))

Start the process
First Presidential debate pulled
Second Presidential debate pulled
Only Vice Presidential debate pulled
Finished in  0.03 seconds


## We now have a DF with the raw 2020 transcript data
# The following code helps standardize the data so they can be appended to the rest of the transcript files

In [11]:
# rev.com includes the time at which each candidate said a given statement
# take this out of the transcript and add it to its own column

t_0 = time.time()
df['Time'] = df['Transcript'].apply(lambda st: st[st.find("(")+1:st.find(")")])

# need to clean up the () that were around the time. Replace with blanks

df['Transcript'] = df['Transcript'].replace("[\(\[].*?[\)\]]", "",regex=True)

# the speaker in the transcript is delimited with ':'; grab the name before
# the colon and put in the Speaker_Clean column. This is similar to the process
# we ran on the CPD data

df['Speaker_Clean'] = df['Transcript'].apply(lambda st: st[:st.find(":")])

# put the purpose as Candidate (even though some of the transcript statements
# are from the moderators, these get cleaned up in the final files, so we can
# call everyone a candidate; others will be removed later)

df['Purpose'] = 'Candidate'

# we will add the Speaker_Standardized column, on which we join a mapping
# file later on, like we did for the CPD transcripts

df['Speaker_standardized'] = df['Speaker_Clean']

# used the following to test that the name pull worked properly; uncomment to create a column that also contains the 
# original transcript lines

#df['Transcript_OG'] = df['Transcript']

# Remove the name in the Transcript rows and use the rest of the text
# (index = 1) to clean up the Transcript. Checked to make sure candidates didn't
# have a colon in their transcript besides the one after their name

df['Transcript'] = df['Transcript'].str.split(':').str[1]

# remove any white space on the Transcript
df['Transcript'] = df['Transcript'].str.strip(' ')

# send the transcripts to a CSV to be used in the rest of the process
df.to_csv('2020debates.csv', index = False)

print('Process complete. Data file created')
print('Finished in {0: .2f} seconds'.format(time.time()-t_0))

Process complete. Data file created
Finished in  0.03 seconds


## 2020debates.csv will be consolidated with the rest of the debate transcripts output from the webscraper in the next file.